# Gradient Boost Regressor Master NB

In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import joblib

import warnings # for ignoring unnecessary warnings
warnings.filterwarnings("ignore")

# ------------------------ #

# 1) Load the dataset 
df = pd.read_excel('sample_data.xlsx') 
# Assume the target variable in the Excel file is named "Target"
# Other variable names are not important here.

# 2) Separate features and target variable from the dataset
X = df.drop(columns=["Target"])
y = df["Target"]

# Separate numeric and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Convert categorical features to strings to avoid mixed types
X[categorical_features] = X[categorical_features].astype(str)

# 3) Create transformers for data preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)) ])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# 4) Create a pipeline with a Gradient Boosting Regressor
base_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# 5) Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6) Train and evaluate the Base Model
base_pipeline.fit(X_train, y_train)

# 7) Predictions and evaluation metrics
print("EVAL METRICS FOR BASE MODEL")
y_train_pred = base_pipeline.predict(X_train)
y_test_pred = base_pipeline.predict(X_test)

def evaluate_model(y_true, y_pred, data_type="Train"):
    print(f"Evaluation metrics for {data_type} data:")
    print(f"Mean Squared Error: {mean_squared_error(y_true, y_pred)}")
    print(f"R2 Score: {r2_score(y_true, y_pred)}")
    print("\n")

evaluate_model(y_train, y_train_pred, "Train")
evaluate_model(y_test, y_test_pred, "Test")

# 8) Hyperparameter Tuning
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__learning_rate': [0.01, 0.1, 0.5, 1.0],
    'regressor__max_depth': [3, 5, 7],
    'regressor__subsample': [0.8, 1.0],
    'regressor__min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(base_pipeline, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print(f"Best Hyperparameters: {grid_search.best_params_}")

# 9) Build and evaluate the Best Model
print("")
print("----EVAL METRICS FOR BEST MODEL----")
print("")

best_pipeline = grid_search.best_estimator_

y_train_best_pred = best_pipeline.predict(X_train)
y_test_best_pred = best_pipeline.predict(X_test)

evaluate_model(y_train_best_pred, y_train, "Train (Best Model)")
evaluate_model(y_test_best_pred, y_test, "Test (Best Model)")

# Compare metrics
metrics_comparison = pd.DataFrame({
    'Metric': ['Mean Squared Error', 'R2 Score'],
    'BaseModlTrain': [mean_squared_error(y_train, y_train_pred), r2_score(y_train, y_train_pred)],
    'BaseModlTest': [mean_squared_error(y_test, y_test_pred), r2_score(y_test, y_test_pred)],
    'BestModTrain': [mean_squared_error(y_train, y_train_best_pred), r2_score(y_train, y_train_best_pred)],
    'BestModlTest': [mean_squared_error(y_test, y_test_best_pred), r2_score(y_test, y_test_best_pred)]
})

print(metrics_comparison)

# 10) Train the Final Model on the entire dataset
final_model = best_pipeline.fit(X, y)

# 11) Save the Final Model
joblib.dump(final_model, 'final_GB_regression_model.pkl')

print(" ")
print("Final model saved as 'final_GB_regression_model.pkl'")

EVAL METRICS FOR BASE MODEL
Evaluation metrics for Train data:
Mean Squared Error: 0.08073159867421878
R2 Score: 0.9969462713195165


Evaluation metrics for Test data:
Mean Squared Error: 0.7261905268554966
R2 Score: 0.9684752724110017


Best Hyperparameters: {'regressor__learning_rate': 0.1, 'regressor__max_depth': 7, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 100, 'regressor__subsample': 0.8}

----EVAL METRICS FOR BEST MODEL----

Evaluation metrics for Train (Best Model) data:
Mean Squared Error: 0.009168130320219226
R2 Score: 0.9996526128983082


Evaluation metrics for Test (Best Model) data:
Mean Squared Error: 0.9107189034054486
R2 Score: 0.9529549457103038


               Metric  BaseModlTrain  BaseModlTest  BestModTrain  BestModlTest
0  Mean Squared Error       0.080732      0.726191      0.009168      0.910719
1            R2 Score       0.996946      0.968475      0.999653      0.960465
 
Final model saved as 'final_GB_regression_model.pkl'
